# <font color = 'pickle'> **HW1  (20 Points):**</font> 

Required Submissions:
1.  Submit  colab/jupyter notebooks only.
2. **Name files as follows : FirstName_hw1**


In [1]:
import torch
import time
import numpy as np
import matplotlib.pyplot as plt

In [1]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: piyushrs. Use `wandb login --relogin` to force relogin


True

In [ ]:
wandb.init(name = "Assignment_1", project = "dl22_l2")

In [2]:
torch.set_printoptions(precision=4, sci_mode=False)

 # <font color = 'pickle'>**Q1 : Create Tensor (1 Point)**
 Create a torch Tensor of shape (5, 3) which is filled with zeros. Modify the tensor to set element (0, 2) to 10 and element (2, 0)  to 100.

In [ ]:
t1 = torch.zeros(5,3)
t1[0,2] = 10
t1[2,0] = 100
t1

tensor([[  0.,   0.,  10.],
        [  0.,   0.,   0.],
        [100.,   0.,   0.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.]])

# <font color = 'pickle'> **Q2: Reshape tensor (1 Point)**
You have following tensor as input:

```x=torch.tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23])```

Using only reshaping functions (like view, reshape, transpose, permute), you need to get at the following tensor as output:

```
tensor([[ 0,  4,  8, 12, 16, 20],
        [ 1,  5,  9, 13, 17, 21],
        [ 2,  6, 10, 14, 18, 22],
        [ 3,  7, 11, 15, 19, 23]])
```



In [ ]:
x=torch.tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23])

In [ ]:
x = x.reshape(6,4)
x = torch.transpose(x, 1, 0)
x

tensor([[ 0,  4,  8, 12, 16, 20],
        [ 1,  5,  9, 13, 17, 21],
        [ 2,  6, 10, 14, 18, 22],
        [ 3,  7, 11, 15, 19, 23]])

# <font color = 'pickle'>**Question 3. Speedtest for vectorization - 2 Points** </font>

Your goal is to measure the speed of linear algebra operations for different levels of vectorization.

1. Construct two matrices $A$ and $B$ with Gaussian random entries of size $1024 \times 1024$. 
1. Compute $C = A B$ using matrix-matrix operations and report the time. (Hint: Use torch.mm)
1. Compute $C = A B$, treating $A$ as a matrix but computing the result for each column of $B$ one at a time. Report the time. (hint use torch.mv inside a for loop)
1. Compute $C = A B$, treating $A$ and $B$ as collections of vectors. Report the time. (Hint: use torch.dot inside nested for loop)

In [ ]:
## Solution 1
torch.manual_seed(42) # dod not chnage this
A = torch.rand(1024, 1024)
B = torch.rand(1024, 1024)

In [ ]:
## Solution 2
start=time.time()

C = torch.mm(A, B)

print("Matrix by matrix: " + str(time.time()-start) + " seconds")

Matrix by matrix: 0.017949819564819336 seconds


In [ ]:
## Solution 3
C= torch.empty(1024,1024)
start = time.time()

for i in B.T:
    C += torch.mv(A, i)

print("Matrix by vector: " + str(time.time()-start) + " seconds") 

Matrix by vector: 0.23163795471191406 seconds


In [ ]:
## Solution 4
C= torch.empty(1024,1024)
start = time.time()

for i in A:
    for j in B:
        C += torch.dot(i, j)
    
print("vector by vector: " + str(time.time()-start) + " seconds")  

vector by vector: 60.51954412460327 seconds


# <font color = 'pickle'>**Question 4 : Redo Question 3 by using GPU - 2 Point**

<font size = 4, color = 'dodgerblue'> **Using GPUs**

How to use GPUs in Google Colab<br>
In Google Colab -- Go to Runtime Tab at top -- select change runtime type -- for hardware accelartor choose GPU

In [ ]:
# Check if GPU is availaible
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [ ]:
## Solution 1
torch.manual_seed(42)
A= torch.randn((1024, 1024),device=device)
B= torch.randn((1024, 1024),device=device)

In [ ]:
## Solution 2
start=time.time()

C = torch.mm(A, B)

print("Matrix by matrix: " + str(time.time()-start) + " seconds")

Matrix by matrix: 1.4516363143920898 seconds


In [ ]:
## Solution 3
C= torch.empty(1024,1024, device = device)
start = time.time()

for i in B.T:
    C += torch.mv(A, i)

print("Matrix by vector: " + str(time.time()-start) + " seconds") 

Matrix by vector: 0.06382942199707031 seconds


In [ ]:
## Solution 4
C= torch.empty(1024,1024, device = device)
start = time.time()

for i in A:
    for j in B:
        C += torch.dot(i, j)

print("vector by vector: " + str(time.time()-start) + " seconds")  

vector by vector: 45.64728617668152 seconds


# <font color = 'pickle'>**Question 5. Memory efficient computation - 2 Points**</font>

We want to compute $C \leftarrow A \cdot B + C$, where $A, B$ and $C$ are all matrices. Implement this in the most memory efficient manner. Pay attention to the following two things:

1. Do not allocate new memory for the new value of $C$.
1. Do not allocate new memory for intermediate results if possible. Hint: If you implement this correcly the memory location of C given by id(C) will be same in both the cells below.

In [ ]:
A= torch.randn((1000, 1000),device=device)
B= torch.randn((1000,1000),device=device)
C= torch.randn((1000, 1000),device=device)
print(id(C))

3043468812800


In [ ]:
torch.cuda.synchronize()
C += torch.mm(A, B)
# C.add_(torch.mm(A, B))
# C.add_(C)

print(id(C))

3043468812800


#<font color = 'pickle'>**Question 6. Broadcast Operations - 2 Points**

In order to perform polynomial fitting we want to compute a design matrix $A$ with 

$$A_{ij} = x_i^j$$

Our goal is to implement this **without a single for loop** entirely using vectorization and broadcast. Here $1 \leq j \leq 3$ and $x = \{1,2,3,4,5\}$. Implement code that generates following A matrix

$$\begin{bmatrix} 1 & 1 & 1 \\ 2 & 4 & 8 \\ 3 & 9 & 27 \\ 4 & 16 & 64 \\ 5 & 25 & 125\end{bmatrix}$$

In [ ]:
x = torch.tensor([1,2,3,4,5])
j = torch.tensor([1,2,3])
x = x.reshape(5,1)
x = x**j
x

tensor([[  1,   1,   1],
        [  2,   4,   8],
        [  3,   9,  27],
        [  4,  16,  64],
        [  5,  25, 125]])

# <font color = 'pickle'>**Q7. Regression from scartch using Pytroch Tensors (5 Points)**</font>

Imagine that you're trying to figure out relationship between two variables x and y . You have some idea but you aren't quite sure yet whether the dependence is linear or quadratic. 

Your goal is to use least mean squares regression to identify the coefficients for the following three models using. The three models are:

1. Quadratic model where $\mathrm{y} = b + w_1 \cdot \mathrm{x} + w_2 \cdot \mathrm{x}^2$.
1. Linear model where $\mathrm{y} = b + w_1 \cdot \mathrm{x}$.
1. Linear model with no bias  where $\mathrm{y} = w_1 \cdot \mathrm{x}$.

- You will use batch gradient descent to estimate the model co-efficients. Batch gradient descent uses complete training data at each iteration. 
- We will implement only training loop (no splitting of data in to training/validation).
- The training loop will have only one for loop. We need to iterate over whole data in each epoch. We do not need to craete batches.
- You may have to try different values of number of epochs/ learning rate to get good results.
- You are not allowed to use Pytorch's nn.module or functions from Pytorch. You will write function for loss function (mean sqaured error), prediction and gradient update from scratch.

## <font color = 'pickle'> **Data**

In [19]:
x = torch.tensor([1.5420291, 1.8935232, 2.1603365, 2.5381863, 2.893443, \
                    3.838855, 3.925425, 4.2233696, 4.235571, 4.273397, \
                    4.9332876, 6.4704757, 6.517571, 6.87826, 7.0009003, \
                    7.035741, 7.278681, 7.7561755, 9.121138, 9.728281])
y = torch.tensor([63.802246, 80.036026, 91.4903, 108.28776, 122.781975, \
                    161.36314, 166.50816, 176.16772, 180.29395, 179.09758, \
                    206.21027, 272.71857, 272.24033, 289.54745, 293.8488, \
                    295.2281, 306.62274, 327.93243, 383.16296, 408.65967])

In [20]:
x.shape

torch.Size([20])

In [12]:
def linear_model(X, w, b):
    return torch.mm(X, w.T) + b

def quadratic_model(X, w, b):
    return torch.mm(X, w.T) + b

def linear_no_bias(X, w):
    return torch.mm(X, w.T)

def loss_function(ypred, y):
    error = ypred - y.view(ypred.shape)
    mse = error.T.mm(error)/len(y)
    return mse

def sgd(params, params_grad, lr):
    for param, param_grad in zip(params, params_grad):
        param -= lr * param_grad

In [48]:
w = torch.normal(0, 0.01, size = (1,1), requires_grad= False)
b = torch.zeros(1, requires_grad=False)

x = x.reshape(-1,1)
y = y.reshape(-1,1)
print(w)
print(x.shape, y.shape)

tensor([[-0.0068]])
torch.Size([20, 1]) torch.Size([20, 1])


In [45]:
lr = 0.005
epochs = 20

for epoch in range(epochs):
    ypred = linear_model(x, w, b)
    batch_loss = loss_function(ypred, y)
    grad_w = 2*x.T.mm(ypred-y)/len(y)
    grad_b = 2*(ypred - y).sum()/len(y)
    sgd([w.T, b], [grad_w, grad_b], lr)
    
    train_l = loss_function(linear_model(x, w, b), y)
    # wandb.log({"/Loss_2_2": train_l.item()})
    
    print(f'epoch {epoch + 1}, loss {float(train_l.item()):f}')

epoch 1, loss 25589.222656
epoch 2, loss 11289.986328
epoch 3, loss 4983.426758
epoch 4, loss 2201.964600
epoch 5, loss 975.214722
epoch 6, loss 434.157288
epoch 7, loss 195.516724
epoch 8, loss 90.255417
epoch 9, loss 43.819752
epoch 10, loss 23.328533
epoch 11, loss 14.279955
epoch 12, loss 10.278070
epoch 13, loss 8.502085
epoch 14, loss 7.707697
epoch 15, loss 7.346410
epoch 16, loss 7.176120
epoch 17, loss 7.090117
epoch 18, loss 7.041326
epoch 19, loss 7.008946
epoch 20, loss 6.983895


Making changes for quadratic model in features and weights.

In [27]:
x = torch.tensor([1.5420291, 1.8935232, 2.1603365, 2.5381863, 2.893443, \
                    3.838855, 3.925425, 4.2233696, 4.235571, 4.273397, \
                    4.9332876, 6.4704757, 6.517571, 6.87826, 7.0009003, \
                    7.035741, 7.278681, 7.7561755, 9.121138, 9.728281])
y = torch.tensor([63.802246, 80.036026, 91.4903, 108.28776, 122.781975, \
                    161.36314, 166.50816, 176.16772, 180.29395, 179.09758, \
                    206.21027, 272.71857, 272.24033, 289.54745, 293.8488, \
                    295.2281, 306.62274, 327.93243, 383.16296, 408.65967])

w = torch.normal(0, 0.01, size = (1,2), requires_grad= False)
b = torch.zeros(1, requires_grad=False)

x = x.reshape(-1,1)
y = y.reshape(-1,1)

x_power = torch.tensor([1,2])

x = x**x_power

print(w)
print(x.shape, y.shape)

tensor([[-0.0067,  0.0060]])
torch.Size([20, 2]) torch.Size([20, 1])


In [28]:
lr = 0.0001
epochs = 10000

for epoch in range(epochs):
    ypred = quadratic_model(x, w, b)
    batch_loss = loss_function(ypred, y)
    grad_w = 2*x.T.mm(ypred-y)/len(y)
    grad_b = 2*(ypred - y).sum()/len(y)
    sgd([w.T, b], [grad_w, grad_b], lr)
    
    train_l = loss_function(quadratic_model(x, w, b), y)
    # wandb.log({"/Loss_2_2": train_l.item()})
    
    print(f'epoch {epoch + 1}, loss {float(train_l.item()):f}')

Streaming output truncated to the last 5000 lines.
epoch 5001, loss 28.876373
epoch 5002, loss 28.854584
epoch 5003, loss 28.832905
epoch 5004, loss 28.811222
epoch 5005, loss 28.789577
epoch 5006, loss 28.767941
epoch 5007, loss 28.746347
epoch 5008, loss 28.724707
epoch 5009, loss 28.703028
epoch 5010, loss 28.681509
epoch 5011, loss 28.659992
epoch 5012, loss 28.638500
epoch 5013, loss 28.617014
epoch 5014, loss 28.595516
epoch 5015, loss 28.574030
epoch 5016, loss 28.552603
epoch 5017, loss 28.531260
epoch 5018, loss 28.509842
epoch 5019, loss 28.488480
epoch 5020, loss 28.467117
epoch 5021, loss 28.445858
epoch 5022, loss 28.424557
epoch 5023, loss 28.403233
epoch 5024, loss 28.382015
epoch 5025, loss 28.360752
epoch 5026, loss 28.339560
epoch 5027, loss 28.318396
epoch 5028, loss 28.297268
epoch 5029, loss 28.276150
epoch 5030, loss 28.254999
epoch 5031, loss 28.233944
epoch 5032, loss 28.212856
epoch 5033, loss 28.191818
epoch 5034, loss 28.170822
epoch 5035, loss 28.149784
epoc

In [49]:
lr = 0.005
epochs = 20

for epoch in range(epochs):
    ypred = linear_no_bias(x, w)
    batch_loss = loss_function(ypred, y)
    grad_w = 2*x.T.mm(ypred-y)/len(y)
    grad_b = 2*(ypred - y).sum()/len(y)
    sgd([w.T, b], [grad_w, grad_b], lr)
    
    train_l = loss_function(linear_no_bias(x, w), y)
    # wandb.log({"/Loss_2_2": train_l.item()})
    
    print(f'epoch {epoch + 1}, loss {float(train_l.item()):f}')

epoch 1, loss 26207.660156
epoch 2, loss 11851.322266
epoch 3, loss 5359.631836
epoch 4, loss 2424.203369
epoch 5, loss 1096.851440
epoch 6, loss 496.646423
epoch 7, loss 225.243683
epoch 8, loss 102.520035
epoch 9, loss 47.026520
epoch 10, loss 21.933361
epoch 11, loss 10.586662
epoch 12, loss 5.455899
epoch 13, loss 3.135878
epoch 14, loss 2.086795
epoch 15, loss 1.612419
epoch 16, loss 1.397915
epoch 17, loss 1.300914
epoch 18, loss 1.257044
epoch 19, loss 1.237203
epoch 20, loss 1.228244


Model with no weight gives the least loss.

# <font color = 'pickle'> **Q8 Image Classification using - Pixel Similarity - 5 Points**

## <font color = 'pickle'> **Import libraries**

In [51]:
# import libraries
import torchvision
import torchvision.transforms as transforms
from pathlib import Path
import cv2 as cv
from google.colab.patches import cv2_imshow # for image display
import pandas as pd

## <font color = 'pickle'> **Mount Google Deive**

In [52]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## <font color = 'pickle'>**Path to Dowanload Data**

<font color = 'dodgerblue'>**Make sure you change the Path to where you want to save data.**</font>
In the code below - **data/datasets** is the folder name in my google drive. You can change this to appropriate folder for your drive. For example you may want to save data to BUAN6341/HW1/Data. In this case the below code should be modified to :

 ```data_folder = Path('/content/drive/MyDrive/BUAN6341/HW2/Data')```

In [53]:
data_folder = Path('/content/drive/MyDrive/Colab Notebooks/Deep Learning/Assignment 1/datasets/')

## <font color = 'pickle'>**Download MNIST training Data**
The MNIST is a dataset of  handwritten digits, available from this page http://yann.lecun.com/exdb/mnist/. It has a training set of 60,000 examples, and a test set of 10,000 examples. In this task we will use images for digits 3 and 7.

In [54]:
trainset = torchvision.datasets.MNIST(root = data_folder,
                                      train = True,
                                      download = True,
                                      )

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /content/drive/MyDrive/Colab Notebooks/Deep Learning/Assignment 1/datasets/MNIST/raw/train-images-idx3-ubyte.gz to /content/drive/MyDrive/Colab Notebooks/Deep Learning/Assignment 1/datasets/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /content/drive/MyDrive/Colab Notebooks/Deep Learning/Assignment 1/datasets/MNIST/raw/train-labels-idx1-ubyte.gz to /content/drive/MyDrive/Colab Notebooks/Deep Learning/Assignment 1/datasets/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /content/drive/MyDrive/Colab Notebooks/Deep Learning/Assignment 1/datasets/MNIST/raw/t10k-images-idx3-ubyte.gz to /content/drive/MyDrive/Colab Notebooks/Deep Learning/Assignment 1/datasets/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /content/drive/MyDrive/Colab Notebooks/Deep Learning/Assignment 1/datasets/MNIST/raw/t10k-labels-idx1-ubyte.gz to /content/drive/MyDrive/Colab Notebooks/Deep Learning/Assignment 1/datasets/MNIST/raw



## <font color = 'pickle'>**Subset of images**

trainset.data has images and trainset.targets has the labels. Below we will create a Boolean mask for digits 3 and 7. We will use this mask to extract subset of images.

In [23]:
# get the mask 
idx3 = trainset.targets==3
idx7 = trainset.targets==7

In [24]:
idx3 # This take the value of True wherever the label was 3 and False otherwise

tensor([False, False, False,  ..., False, False, False])

In [25]:
threes = trainset.data[idx3] # use Boolean mask to extract images for digit three
sevens = trainset.data[idx7] # use Boolean mask to extract images for digit seven

In [26]:
threes.shape, sevens.shape

(torch.Size([6131, 28, 28]), torch.Size([6265, 28, 28]))

In [27]:
type(threes[0])

torch.Tensor

In [28]:
# get a sample image
sample_img_3 = threes[0]

In [29]:
sample_img_3.shape

torch.Size([28, 28])

### <font color = 'pickle'>**Task1**
Reshape the image so that it has following  shape: 28, 28, 1. Then convert the tensor to numpy array. 

In [30]:
sample_img_3_numpy =  sample_img_3.reshape(28,28,1).numpy()

In [31]:
# let us look at the image
# the above steps were needed because the function cv2_imshow needs a three dimenional numpy array of the 
# shape H x W  X C (height, width, number of channels). The number of channels for black and white images is 1.
cv2_imshow(sample_img_3_numpy)

### <font color = 'pickle'>**Task2**

Repeat the above steps to show the first image for digit 7.
 - Get the first image.
 - Reshape the image to 28, 28, 1
 - Convert the tensor to numpy array


In [32]:
# code here
sample_img_7 = sevens[0]
sample_img_7_np = sample_img_7.reshape(28,28,1).numpy()
cv2_imshow(sample_img_7_np)

## <font color = 'pickle'>**Visualizing images as pixel values**

<font color = 'dodgerblue'> 
- In a computer, images are stored in the form of matrices. The numbers in the matrix are called "pixel values."
- These pixel values show how bright each pixel is.
- 0 means "black," and 255 means "white."
- The matrix of numbers is called the channel, and there is only one channel in a grayscale image.
- For color inages , we have three channels(RGB - Red, Green Blue). That is we need three matrices to store color images.
</font>

In [33]:
df = pd.DataFrame(sample_img_3[5:24, 7:22].numpy())
df.style.set_properties(**{'font-size':'8pt'})

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0,0,0,0,38,43,105,255,253,253,253,253,253,174,6
1,0,0,43,139,224,226,252,253,252,252,252,252,252,252,158
2,0,0,178,252,252,252,252,253,252,252,252,252,252,252,252
3,0,0,109,252,252,230,132,133,132,132,189,252,252,252,252
4,0,0,4,29,29,24,0,0,0,0,14,226,252,252,172
5,0,0,0,0,0,0,0,0,0,0,85,243,252,252,144
6,0,0,0,0,0,0,0,0,0,88,189,252,252,252,14
7,0,0,0,0,0,0,0,91,212,247,252,252,252,204,9
8,0,0,32,125,193,193,193,253,252,252,252,238,102,28,0
9,0,45,222,252,252,252,252,253,252,252,252,177,0,0,0


<font color = 'dodgerblue'> - In the matrix below we have flipped color just for illustartion (We are using 0 for white instead of black).

In [34]:
df.style.set_properties(**{'font-size':'8pt'}).background_gradient('Greys')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0,0,0,0,38,43,105,255,253,253,253,253,253,174,6
1,0,0,43,139,224,226,252,253,252,252,252,252,252,252,158
2,0,0,178,252,252,252,252,253,252,252,252,252,252,252,252
3,0,0,109,252,252,230,132,133,132,132,189,252,252,252,252
4,0,0,4,29,29,24,0,0,0,0,14,226,252,252,172
5,0,0,0,0,0,0,0,0,0,0,85,243,252,252,144
6,0,0,0,0,0,0,0,0,0,88,189,252,252,252,14
7,0,0,0,0,0,0,0,91,212,247,252,252,252,204,9
8,0,0,32,125,193,193,193,253,252,252,252,238,102,28,0
9,0,45,222,252,252,252,252,253,252,252,252,177,0,0,0


### <font color = 'pickle'>**Task3**

<font color = 'pickle'>**Convert threes and sevens generated earlier to float and and take the mean along dimension 0.**

Hint(You can use tensor.float() and then take the mean along dimesnion 0.

Here we are generating an average image by taking the average of each pixel across images.

In [35]:
mean_threes =  torch.mean(torch.Tensor.float(threes), dim = 0)
mean_sevens =  torch.mean(torch.Tensor.float(sevens), dim = 0)

In [36]:
mean_threes.shape
mean_sevens.shape

torch.Size([28, 28])

In [37]:
cv2_imshow(mean_threes.unsqueeze(dim=2).numpy())
print()
cv2_imshow(mean_sevens.unsqueeze(dim=2).numpy())

## <font color = 'pickle'>**Prediction for validation images**

We will calculate the distance of each validation image from the mean_threes and mean_sevens. If the validation image is closest to mean_threes then we will predict 1 else we will predict 0.

## <font color = 'pickle'>**Get Valid Dataset**

In [6]:
validset = torchvision.datasets.MNIST(root = data_folder,
                                      train = False,
                                      download = True,
                                      )

In [38]:
# get the mask 
idx3 = validset.targets==3
idx7 = validset.targets==7

In [39]:
validset.data.shape

torch.Size([10000, 28, 28])

In [41]:
# get images and labels using the mask
valid_data_3_7 = validset.data[idx3+idx7]
valid_targets_3_7 = validset.targets[idx3 + idx7]

In [42]:
# check the shape of the inputs (images)
# we have 2038 images with size 28 x 28.
valid_data_3_7.shape

torch.Size([2038, 28, 28])

In [43]:
# lables for valid dataset
valid_targets_3_7

tensor([7, 7, 3,  ..., 3, 7, 3])

In [44]:
# change the lable to 1 where the label was 3
valid_targets_3_7[valid_targets_3_7==3] = 1

In [45]:
# change the lable to 0 where the label was 7
valid_targets_3_7[valid_targets_3_7==7] = 0

In [46]:
valid_targets_3_7.unique()

tensor([0, 1])

In [47]:
valid_targets_3_7

tensor([0, 0, 1,  ..., 1, 0, 1])

In [68]:
valid_data_3_7.shape

torch.Size([2038, 28, 28])

## <font color = 'pickle'>**Distance between images**

### <font color = 'pickle'>**Task4** 

<font color = 'pickle'>**Write a function to calculate the distance between two images a and b.** 
 - Calculate the difference between corresponding pixels of two images.
 - Calculate square of differences.
 - Take the mean (take the mean across last two dimensions (-1, -2) of the square of the differences and then take the square root.

In [73]:
def dist_images(a, b):
  diff = ((a - b)**2)
  mean_diff = torch.mean(diff, dim=(-1,-2))
  return mean_diff**0.5

In [74]:
# calculate the distance of the each image in validation set from mean_threes
# bacause of the  broadcasting we are able to use the above function which was written to 
# calculate the distnace between two images
valid_3_dist = dist_images(valid_data_3_7, mean_threes)
valid_3_dist

tensor([70.2972, 71.7095, 79.9791,  ..., 59.1907, 68.7125, 58.1954])

In [75]:
# calculate the distance of the each image in validation set from mean_sevens
valid_7_dist = dist_images(valid_data_3_7, mean_sevens)

In [76]:
# if you have done everything right, you will observe following shapes.
valid_3_dist.shape, valid_7_dist.shape

(torch.Size([2038]), torch.Size([2038]))

## <font color = 'pickle'>**Prediction Using Distance**

### <font color = 'pickle'>**Task5**



In [78]:
# step 1 : generate the tensor filled with zeros taht has the same shape as valid_target_3_7
prediction =  torch.zeros_like(valid_targets_3_7)

In [82]:
# step 2: update the tensor prediction. Update the value to 1, when valid_3_dist < valid_7_dist.
# Hint (Generate a Boolean mask using the condition valid_3_dist < valid_7_dist and use the mask to update value to 1 )

for i in range(len(valid_3_dist)):
  if valid_3_dist[i] < valid_7_dist[i]:
    prediction[i] = 1

In [83]:
prediction

tensor([0, 0, 1,  ..., 1, 0, 1])

## <font color = 'pickle'>**Accuracy**

In [84]:
correct = prediction == valid_targets_3_7

In [85]:
correct

tensor([True, True, True,  ..., True, True, True])

In [86]:
100 * correct.float().mean().item()

96.6143250465393